## **Check for activity and text change variable**
- non_production action time / total _action time ratio has no correlation with score
- input action time / total action time ratio has no correlation with score
- log productive time has a better correlation with scores. 0.57
- log input-remove-cut action time has a correlation of 0.58 with scores.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from dask.distributed import Client
import dask.dataframe as dd

In [2]:
notes = set() 
input_folder = "kaggle/input/linking-writing-processes-to-writing-quality/"

In [3]:
train_logs = pd.read_csv(input_folder + "train_logs.csv",delimiter = ",",header = 0)
train_scores = pd.read_csv(input_folder +"train_scores.csv", delimiter = ",", header = 0)
scores = pd.Series(data = train_scores['score'].values, index = train_scores['id'].values, name = 'score')

In [4]:
train_logs.head(5)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [5]:
train_logs.activity.unique()

array(['Nonproduction', 'Input', 'Remove/Cut', 'Replace',
       'Move From [284, 292] To [282, 290]',
       'Move From [287, 289] To [285, 287]',
       'Move From [460, 461] To [465, 466]', 'Paste',
       'Move From [905, 1314] To [907, 1316]',
       'Move From [565, 743] To [669, 847]',
       'Move From [669, 847] To [565, 743]',
       'Move From [1041, 1121] To [1496, 1576]',
       'Move From [1455, 1557] To [1323, 1425]',
       'Move From [2268, 2275] To [2247, 2254]',
       'Move From [213, 302] To [902, 991]',
       'Move From [0, 158] To [234, 392]',
       'Move From [460, 465] To [925, 930]',
       'Move From [810, 906] To [816, 912]',
       'Move From [186, 187] To [184, 185]',
       'Move From [140, 272] To [299, 431]',
       'Move From [114, 140] To [272, 298]',
       'Move From [1386, 1450] To [1445, 1509]',
       'Move From [442, 524] To [296, 378]',
       'Move From [408, 414] To [390, 396]',
       'Move From [1144, 1147] To [1142, 1145]',
       'Move 

In [6]:
train_logs.head(4)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1


In [7]:
total_action_time = train_logs.groupby('id')['action_time'].agg('sum')
input_action_time = train_logs[train_logs['activity'] == "Input"].groupby('id')['action_time'].agg('sum')
non_prod_action_time = train_logs[train_logs['activity'] == "Nonproduction"].groupby('id')['action_time'].agg('sum')
remove_cut_action_time =train_logs[train_logs['activity'] == "Remove/Cut"].groupby('id')['action_time'].agg('sum')
paste_action_time = train_logs[train_logs['activity'] == "Paste"].groupby('id')['action_time'].agg('sum')
replace_action_time =train_logs[train_logs['activity'] == "Replace"].groupby('id')['action_time'].agg('sum') 
move_action_time = total_action_time - input_action_time - non_prod_action_time - remove_cut_action_time
move_action_time -= replace_action_time
# text change
# no_text_change = train_logs[train_logs['text_change'] == "NoChange"].groupby('id')['action_time'].agg('sum')
# combination:
combo_action_time = np.log(input_action_time + remove_cut_action_time + paste_action_time) * np.log(total_action_time)
productive_time = np.log(total_action_time - non_prod_action_time)

In [8]:
input_action_time.name = "input_action_time"
non_prod_action_time.name = "non_prod_action_time"
remove_cut_action_time.name = "remove_cut_action_time"
paste_action_time.name = "paste_action_time"
combo_action_time.name = "combo_action_time"
productive_time.name = "productive_time"
move_action_time.name = "move_action_time"

In [13]:
 z = pd.merge(input_action_time, non_prod_action_time, left_index = True, right_index = True)
# z = pd.merge(z, remove_cut_action_time, left_index = True, right_index = True)
# z = pd.merge(z, replace_action_time, left_index = True, right_index = True)
z = pd.merge(z, move_action_time, left_index = True, right_index = True)
z = pd.merge(z, paste_action_time, left_index = True, right_index = True)
z = pd.merge(z, scores, left_index = True, right_index = True)
z.corr()

,input_action_time,non_prod_action_time,move_action_time,paste_action_time,score
input_action_time,1.000000,0.009311,0.025130,-0.049071,0.489034
non_prod_action_time,0.009311,1.000000,0.006552,-0.050040,0.126118
move_action_time,0.025130,0.006552,1.000000,0.942911,-0.025780
paste_action_time,-0.049071,-0.050040,0.942911,1.000000,-0.085053
score,0.489034,0.126118,-0.025780,-0.085053,1.000000


,productive_time,combo_action_time,move_action_time,paste_action_time,score
productive_time,1.000000,0.987920,-0.009824,-0.082893,0.524820
combo_action_time,0.987920,1.000000,-0.007520,-0.090679,0.536816
move_action_time,-0.009824,-0.007520,1.000000,0.942911,-0.025780
paste_action_time,-0.082893,-0.090679,0.942911,1.000000,-0.085053
score,0.524820,0.536816,-0.025780,-0.085053,1.000000


In [11]:
train_logs.text_change.unique()

array(['NoChange', 'q', ' ', ..., 'qq qqq qqqq qqqqq',
       'qq qqqqq qqqq qq qqqqq ',
       '\n qqqqq qqqqqq qqqqqqqqqq qq q qqqqqqqq qqq qqq qqqq qqqqqq q qqq. \n\nqqqq qqq qq qqqqqqq qqq:\n- \n- qqq qqqqqqq qqqq q qqqqqq qqqqqqqq qq qqqq qqqqqqqq '],
      dtype=object)